## Importiing libs

In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

## Building data

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import torch

%matplotlib inline

np.random.seed(2)

T = 20
L = 1000
N = 100

x = np.empty((N, L), 'int64')
x[:] = np.array(range(L)) + np.random.randint(-4 * T, 4 * T, N).reshape(N, 1)
data = np.sin(x / 1.0 / T).astype('float64')


plt.figure(figsize=(15,5))
plt.title('Predict future values for time sequences', fontsize=20)
plt.xlabel('x', fontsize=20)
plt.ylabel('y', fontsize=20)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(data[0], 'r')
plt.plot(data[1], 'g')
plt.plot(data[2], 'b')

## Setting Architecture

* Implemente um modelo para predição de valores futuros de uma sequência usando apenas *LSTMCell* e *Linear*
* Cada entrada (caracter) possui dimensão (1): um ponto na série temporal
* *Hidden size* possui dimensão (128): hiperparâmetro 
* Saída possui dimensão (1): próximo ponto na série temporal
* Batch size = input_data.size(0) de acordo com a implementação do loop de treinamento

**Links úteis**

LSTMCell: https://pytorch.org/docs/stable/nn.html#torch.nn.LSTMCell


In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable

# Recurrent neural network 
class CustomNetwork(nn.Module):
    
    def __init__(self, cell_type="lstm", input_size=1, hidden_size=128, output_size=1, nonlinearity="tanh"):
        
        super(CustomNetwork, self).__init__()
        
             

    def forward(self, input_data, future=0):

        # Set initial hidden and cell states 
        
        # Loop to feed the network one time step at a time
        for input_t in input_data.chunk(input_data.size(1), dim=1):
        
        # If predicting the future (test time)
        for i in range(future):  
            
# Build the model
#seq = CustomNetwork(cell_type='lstm').double()
seq = CustomNetwork(cell_type='lstm').double().cuda() #GPU

## Setting Loss and Optimizer

In [0]:
import torch.optim as optim

#criterion = nn.MSELoss()
criterion = nn.MSELoss().cuda() #GPU

# use LBFGS as optimizer since we can load the whole data to train
optimizer = optim.LBFGS(seq.parameters(), lr=0.8)

## Train / Test Model

In [0]:
#input = Variable(torch.from_numpy(data[3:, :-1]).double())
#target = Variable(torch.from_numpy(data[3:, 1:]).double())
#test_input = Variable(torch.from_numpy(data[:3, :-1]).double())
#test_target = Variable(torch.from_numpy(data[:3, 1:]).double())

#GPU
input = Variable(torch.from_numpy(data[3:, :-1]).double()).cuda() 
target = Variable(torch.from_numpy(data[3:, 1:]).double()).cuda()
test_input = Variable(torch.from_numpy(data[:3, :-1]).double()).cuda()
test_target = Variable(torch.from_numpy(data[:3, 1:]).double()).cuda()

# Train the model
num_epochs=7
for epoch in range(num_epochs):
  
    # Set to Train mode
    seq.train()
    
    # Forward and Backward pass
    def closure():
      optimizer.zero_grad()
      out = seq(input)
      loss = criterion(out, target)
      loss.backward()
      return loss
    optimizer.step(closure)
    
    # Test the model
    
    # Set to eval mode
    seq.eval()
    
    future = 1000
    pred = seq(test_input, future=future)
    loss = criterion(pred[:, :-future], test_target)
    print ('Epoch [{}/{}], Loss: {:.6f}' 
                   .format(epoch+1, num_epochs, loss.data[0]))

# Draw the result
y = pred.data.cpu().numpy()
plt.figure(figsize=(25,5))
plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=20)
plt.xlabel('x', fontsize=20)
plt.ylabel('y', fontsize=20)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
def draw(yi, color):
    plt.plot(np.arange(input.size(1)), yi[:input.size(1)], color, linewidth = 2.0)
    plt.plot(np.arange(input.size(1), input.size(1) + future), yi[input.size(1):], color + ':', linewidth = 2.0)
draw(y[0], 'r')
draw(y[1], 'g')
draw(y[2], 'b')
    